In [156]:
import pandas as pd
from pandasql import sqldf

query = lambda q: sqldf(q, globals())

caixas = pd.read_csv('../data/caixas.csv')
estoque = pd.read_csv('../data/estoque.csv')

In [157]:
cmd = f"""
    SELECT DISTINCT 
        caixas.CAIXA_ID, 
        MIN(estoque.ANDAR) AS ANDAR, 
        estoque.SKU,
        SUM(caixas.PECAS) AS PECAS
    FROM caixas
    JOIN estoque
        ON caixas.SKU = estoque.SKU
        AND caixas.PECAS <= estoque.PECAS
    GROUP BY caixas.CAIXA_ID, estoque.SKU"""
caixas_andares = query(cmd)
print(caixas_andares)

       CAIXA_ID  ANDAR        SKU  PECAS
0            12      1      SKU_1     10
1            12      2     SKU_10      6
2            12      0     SKU_11      3
3            12      0     SKU_12      4
4            12      2     SKU_13     18
...         ...    ...        ...    ...
22065      6938      0  SKU_12898      2
22066      6939      0  SKU_12756     10
22067      6939      0  SKU_12898      3
22068      6940      0  SKU_12755      1
22069      6940      0  SKU_12756     10

[22070 rows x 4 columns]


In [158]:
cmd = f"""
    SELECT 
        CAIXA_ID, 
        COUNT(SKU) AS SKUS, 
        SUM(PECAS) AS PECAS,
        COUNT(DISTINCT ANDAR) AS ANDARES
    FROM caixas_andares
    GROUP BY CAIXA_ID""" 
caixas_info = query(cmd)
print(caixas_info)

      CAIXA_ID  SKUS  PECAS  ANDARES
0           12    17    238        3
1           13    31    333        3
2           45    27    229        3
3           46    26    200        3
4           47    17    221        1
...        ...   ...    ...      ...
2067      6936     4    119        1
2068      6937     7     91        1
2069      6938     2      8        1
2070      6939     2     13        1
2071      6940     2     11        1

[2072 rows x 4 columns]


parâmetros de priorização:
1. menor quantidade de andares necessários
2. menor quantidade total de peças
3. andar mais baixo

In [159]:
cmd = f"""
    SELECT 
        caixas_info.CAIXA_ID, 
        caixas_andares.ANDAR, 
        caixas_andares.SKU, 
        caixas_andares.PECAS
    FROM caixas_andares
    JOIN caixas_info
        ON caixas_andares.CAIXA_ID = caixas_info.CAIXA_ID
    ORDER BY 
        caixas_info.ANDARES ASC,
        caixas_info.PECAS ASC, 
        caixas_andares.ANDAR ASC""" # parâmetros de priorização de caixas no ORDER BY
caixas_andares = query(cmd)
print(caixas_andares)

       CAIXA_ID  ANDAR        SKU  PECAS
0           612      0  SKU_19448      1
1           613      0  SKU_19448      1
2           614      0  SKU_19448      1
3           616      0  SKU_19448      1
4           617      0  SKU_19448      1
...         ...    ...        ...    ...
22065       704      1    SKU_348      2
22066       704      1    SKU_349      3
22067       704      2    SKU_144      4
22068       704      2    SKU_244      6
22069       704      2    SKU_288      2

[22070 rows x 4 columns]


In [162]:
for caixa, andar in caixas_andares[['CAIXA_ID', 'ANDAR']].values:
    pass